## Introduction

In this jupyter notebook, we train an ReID model using the Resnet50 architecture as our feature generator. For this task we employ the package `torchreid` to construct our model and data loader. We train a total of 60 epochs at a learning rate of 0.003.





In order to run the notebook, first mount your google drive and upload the `market1501` dataset ([link](https://www.kaggle.com/datasets/pengcw1/market-1501)) following the tree structure below.




```
<root>
└───<parent-directory>
    │   ...
    │   ...     
    └───reid-data
    │   └───market1501
    │       └───Market-1501-v15.09.15
    │       │   └───bounding_box_test
    │       │   └───bounding_box_train
    │       │   └───query
```

## Data paths

In [1]:
# change your root path

path = "/content/drive/MyDrive"
import os
from google.colab import drive
os.chdir(path)
os.listdir(path)

['NeurIPS 2022.gsheet',
 'Meet Recordings',
 'poverty-mapping',
 'Colab Notebooks',
 'Poverty Mapping: Results.gdoc',
 'poverty-mapping-literature',
 'Kim, Moses - Regularization Self-Evaluation',
 'vhm-ysmb-kkf - Aug 24, 2022.gjam',
 'FunML',
 'poverty-mapping-data',
 'ML Walk-In Clinic.gslides',
 'ML Walk-In Clinic: Consulting Best Practices and Standards.gdoc',
 'ML Walk-In Clinic',
 'Untitled document.gdoc',
 'playground',
 'big-ml-team',
 'Poverty Mapping in SEA: Rollout Challenges.gdoc',
 'ML_20220311_December.csv.zip',
 'psa',
 'powerpoint',
 'Resume: Moses Kim.gdoc',
 'Learning',
 'Market-1501-v15.09.15.zip',
 'Person_reID_baseline_pytorch',
 'reid',
 'reid-data',
 'deep-person-reid']

## `torchreid` installation

In [2]:
repo_dir_path = os.path.join(path, "deep-person-reid")

if not os.path.exists(repo_dir_path):
    repo_path = "https://github.com/KaiyangZhou/deep-person-reid.git"
    os.system(f"git clone {repo_path}")

In [3]:
# cd into repo 

%cd /content/drive/MyDrive/deep-person-reid/

/content/drive/MyDrive/deep-person-reid


In [4]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
!pip install torch torchvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


The following cell will take a few minutes to execute...

In [6]:
!python setup.py develop

running develop
running egg_info
writing torchreid.egg-info/PKG-INFO
writing dependency_links to torchreid.egg-info/dependency_links.txt
writing requirements to torchreid.egg-info/requires.txt
writing top-level names to torchreid.egg-info/top_level.txt
adding license file 'LICENSE'
writing manifest file 'torchreid.egg-info/SOURCES.txt'
running build_ext
copying build/lib.linux-x86_64-3.8/torchreid/metrics/rank_cylib/rank_cy.cpython-38-x86_64-linux-gnu.so -> torchreid/metrics/rank_cylib
Creating /usr/local/lib/python3.8/dist-packages/torchreid.egg-link (link to .)
torchreid 1.4.0 is already the active version in easy-install.pth

Installed /content/drive/MyDrive/deep-person-reid
Processing dependencies for torchreid==1.4.0
Searching for imageio==2.9.0
Best match: imageio 2.9.0
Adding imageio 2.9.0 to easy-install.pth file
Installing imageio_download_bin script to /usr/local/bin
Installing imageio_remove_bin script to /usr/local/bin

Using /usr/local/lib/python3.8/dist-packages
Searching

In [7]:
# check

import torchreid

In [8]:
# now that we have successfully installed torchreid
# change to dir where we save our logs
root_dir = os.path.join(path, "reid")

if not os.path.exists(root_dir):
    os.makedirs(root_dir)

os.chdir(root_dir)

## Train

In [9]:
# this must be changed to 
# location of data directory in your google drive

# IMAGE
# for me it's the following
image_data_path = os.path.join(
    path,
    "Learning",
    "reid-data"
)


In [10]:
datamanager = torchreid.data.ImageDataManager(
    root=image_data_path,
    sources='market1501',
    height=256,
    width=128,
    batch_size_train=32,
    batch_size_test=100
)

model = torchreid.models.build_model(
    name='resnet50',
    num_classes=datamanager.num_train_pids,
    loss='softmax'
)

model = model.cuda()
optimizer = torchreid.optim.build_optimizer(
    model, optim='adam', lr=0.0003
)
scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler='single_step',
    stepsize=20
)
engine = torchreid.engine.ImageSoftmaxEngine(
    datamanager, model, optimizer, scheduler=scheduler
)
engine.run(
    max_epoch=60,
    save_dir='log/resnet50-softmax-market1501',
    print_freq=10
)

Building train transforms ...
+ resize to 256x128
+ random flip
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset
=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    |   750 |     3368 |         6
  gallery  |   751 |    15913 |         6
  ----------------------------------------
=> Loading test (target) dataset


/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:563: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    |   750 |     3368 |         6
  gallery  |   751 |    15913 |         6
  ----------------------------------------


  **************** Summary ****************
  source            : ['market1501']
  # source datasets : 1
  # source ids      : 751
  # source images   : 12936
  # source cameras  : 6
  target            : ['market1501']
  *****************************************




Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

=> Start training


/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:563: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


epoch: [1/60][10/404]	time 6.233 (8.478)	data 5.976 (7.551)	eta 2 days, 9:03:43	loss 6.9917 (6.7678)	acc 0.0000 (0.0000)	lr 0.000300
epoch: [1/60][20/404]	time 0.195 (6.806)	data 0.000 (6.230)	eta 1 day, 21:47:30	loss 6.6239 (6.8114)	acc 0.0000 (0.1562)	lr 0.000300
epoch: [1/60][30/404]	time 9.617 (7.090)	data 9.330 (6.625)	eta 1 day, 23:40:51	loss 6.5969 (6.7629)	acc 0.0000 (0.1042)	lr 0.000300
epoch: [1/60][40/404]	time 0.197 (6.610)	data 0.000 (6.205)	eta 1 day, 20:26:09	loss 6.6901 (6.7449)	acc 0.0000 (0.0781)	lr 0.000300
epoch: [1/60][50/404]	time 12.578 (6.831)	data 12.291 (6.458)	eta 1 day, 21:53:56	loss 6.6325 (6.7196)	acc 0.0000 (0.0625)	lr 0.000300
epoch: [1/60][60/404]	time 0.200 (6.541)	data 0.000 (6.193)	eta 1 day, 19:55:53	loss 6.6076 (6.6994)	acc 0.0000 (0.1562)	lr 0.000300
epoch: [1/60][70/404]	time 10.199 (6.671)	data 9.911 (6.337)	eta 1 day, 20:47:12	loss 6.5243 (6.6886)	acc 0.0000 (0.1339)	lr 0.000300
epoch: [1/60][80/404]	time 0.201 (6.497)	data 0.000 (6.176)	eta 1 

In [11]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [13]:
os.listdir('log/resnet50-softmax-market1501/model/')  # the weights are here

['model.pth.tar-60']

In [15]:
# load necessary modules

from torchreid import utils
import torch

In [19]:
# model checkpoint
path_to_model = "log/resnet50-softmax-market1501/model"
model_filename = "model.pth.tar-60"

checkpoint_path = os.path.join(
    path_to_model,
    model_filename
)

# load trained model
utils.load_pretrained_weights(model, checkpoint_path)


Successfully loaded pretrained weights from "log/resnet50-softmax-market1501/model/model.pth.tar-60"


In [21]:
# evaluate on sample
model.eval()

# image sample
market_data_loader = datamanager.train_loader
sample_batch = next(iter(market_data_loader))['img']

sample = torch.unsqueeze(sample_batch[0], 0).to("cuda")

sample_features = model(sample)
sample_features.shape

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:563: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


torch.Size([1, 2048])